# Experiments Information

### Data

Dataset split: shuffle by seed=42, then split

* Source Activities: Downstairs, Jogging, Upstairs, Walking

    * Train: 80%
    * Val: 10%
    * Test: 10%

* Target Activites: Sitting, Standing

    * Train: 80%
    * Val: 10%
    * Test: 10%

# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import warnings

from copy import deepcopy
from google.colab import drive
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix, roc_auc_score
from tabulate import tabulate

In [ ]:
# Supress warnings
warnings.resetwarnings()
warnings.simplefilter('ignore', pd.core.common.SettingWithCopyWarning)
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore')

# Global Variables

In [ ]:
data = {}
data_dir = "/content/drive/MyDrive/Classes/CSCE 5280 AI for Wearables/Group Project 2/Datasets/"    # Mica
dirs = {"source": data_dir + "UCI_UC_k_means_preprocessed/", "target": data_dir + "WISDM_UC_k_means_preprocessed/"}
source_labels = set()
target_labels = set()

# Load Data

In [ ]:
if not os.path.exists("drive/"):
    drive.mount('/content/drive')

In [ ]:
# View shapes
def view_data_shapes(data):
    for k1 in data:
        print(k1, "=============================================")
        for k2 in data[k1]:
            print(k2, "---------------------------------------------")
            for k3 in data[k1][k2]:
                print(k3, " ", data[k1][k2][k3].shape)
            print()
        print()

In [ ]:
# Iterate
for k1 in ["source", "target"]:
    data.update({k1: {}})
    for k2 in ["train", "val", "test"]:
        data[k1].update({k2: {}})
        for k3 in ["x", "y"]: #, "x_shaped"]:

            # Get source and target files
            f_name = dirs[k1] + k1 + "_" + k2 + "_" + k3
            x = None
            if k3 == "x_shaped":
                with open(f_name + ".pickle", "rb") as file:
                    x = pickle.load(file)
            else:
                x = pd.read_csv(f_name + ".csv")
            data[k1][k2].update({k3: x})

            # Get label values
            if k3 == "y":
                for x in list(data[k1][k2][k3]["label"].unique()):
                    if k1 == "source":
                        source_labels.add(x)
                    elif k1 == "target":
                        target_labels.add(x)

view_data_shapes(data)

source =============================================
train ---------------------------------------------
x   (16478, 561)
y   (16478, 1)

val ---------------------------------------------
x   (2060, 561)
y   (2060, 1)

test ---------------------------------------------
x   (2060, 561)
y   (2060, 1)


target =============================================
train ---------------------------------------------
x   (3819, 405)
y   (3819, 1)

val ---------------------------------------------
x   (477, 405)
y   (477, 1)

test ---------------------------------------------
x   (478, 405)
y   (478, 1)




# Models

## Single Model Handler

In [ ]:
class ModelHandler():
    
    def __init__(self, name, source_labels, target_labels, checkpoint_path="checkpoints/", checkpoint_file_name="cp.ckpt"):
        self.model_name = name
        self.source_lables = source_labels
        self.target_labels = target_labels
        self.checkpoint_path = checkpoint_path
        self.checkpoint_file_name = checkpoint_file_name
        self.cp_paths = {}
        self.cp_callbacks = {}
        self.hist = {}
            
    # Build model ==============================================================
    def build_model(self, model_type, input_shape, n_labels=6, learning_rate=0.001, metrics=["accuracy"]):
        if model_type not in ["dense", "conv"]:
            print(model_type, "is an invalid model type. Please enter a value in [\"dense\", \"conv\"].")
        self.model_type = model_type
        self.input_shape = input_shape
        self.n_labels = n_labels
        self.learning_rate = learning_rate
        self.metrics = metrics

        model = tf.keras.Sequential([tf.keras.layers.Input(shape=input_shape)])
        
        # Build dense model
        if model_type == "dense":
            model = self.build_dense_model(model)

        # Build cnn model
        elif model_type == "conv":
            model = self.build_conv_model(model, input_shape)

        model.add(tf.keras.layers.Dense(n_labels, activation="softmax"))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                      metrics=metrics
                      )
        self.model = model
    
    def build_transfer_model(self, train_layers, input_shape, n_labels=6, metrics=["accuracy"]):
        tmp = tf.keras.Sequential([tf.keras.layers.Input(shape=input_shape)])
        
        # Build dense model
        if self.model_type == "dense":
            tmp = self.build_dense_model(tmp)

        # Build cnn model
        elif self.model_type == "conv":
            tmp = self.build_conv_model(tmp, input_shape)

        tmp.add(tf.keras.layers.Dense(n_labels, activation="softmax"))

        for new_layer, layer in zip(tmp.layers[1:-train_layers], self.model.layers[1:-train_layers]):
            new_layer.set_weights(layer.get_weights())
            new_layer.trainable = False
            
        tmp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                    metrics=metrics
                    )
        
        self.model = deepcopy(tmp)
    
    def build_dense_model(self, model):
        model.add(tf.keras.layers.Dense(1024, activation="relu"))
        model.add(tf.keras.layers.Dense(512, activation="relu"))
        model.add(tf.keras.layers.Dense(256, activation="relu"))
        model.add(tf.keras.layers.Dense(128, activation="relu"))
        model.add(tf.keras.layers.Dense(64, activation="relu"))
        model.add(tf.keras.layers.Dense(32, activation="relu"))
        model.add(tf.keras.layers.Dense(16, activation="relu"))
        # model.add(tf.keras.layers.Dense(405, activation="relu"))
        # model.add(tf.keras.layers.Dense(202, activation="relu"))
        # model.add(tf.keras.layers.Dense(101, activation="relu"))
        return model

    def build_conv_model(self, model, input_shape):
        model.add(tf.keras.layers.Conv2D(32, kernel_size=(1, 1),
                                                  activation='relu',
                                                  input_shape=input_shape
                                                  ))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
        return model
        
    # Train model ==============================================================

    # Checkpointing ------------------------------------------------------------
    def setup_checkpointing(self, experiment_level):

        # Paths
        if experiment_level not in self.cp_paths:
            self.cp_paths.update({experiment_level: "%s%s"%(self.checkpoint_path, self.checkpoint_file_name)})
        if not os.path.exists(self.cp_paths[experiment_level]):
            os.makedirs(self.cp_paths[experiment_level])

        # Callbacks
        if experiment_level not in self.cp_callbacks:
            self.cp_callbacks.update({experiment_level: tf.keras.callbacks.ModelCheckpoint(filepath=self.cp_paths[experiment_level],
                                                                                           save_weights_only=True,
                                                                                           verbose=1, 
                                                                                           save_freq="epoch",
                                                                                           save_best_only=True
                                                                                           )
                                      })
    
    # Training -----------------------------------------------------------------
    def train_model(self, experiment_level, train_x, train_y, val_x, val_y, epochs=10):

        # Setup checkpoints
        self.setup_checkpointing(experiment_level)

        # Update history dictionary
        if experiment_level not in self.hist:
            self.hist.update({experiment_level: None})

        # Train
        self.hist[experiment_level] = self.model.fit(train_x, train_y, 
                                                     validation_data=(val_x, val_y), 
                                                     epochs=epochs, 
                                                     callbacks=[self.cp_callbacks[experiment_level]]
                                                     )
        
        # Load best
        self.model.load_weights(self.cp_paths[experiment_level])

    # Evaluate =================================================================
    def evaluate(self, test_x, test_y, verbose=True):
        preds =  self.model.predict(test_x)
        preds_select = np.apply_along_axis(np.argmax, 1, preds)
        true = [y[0] for y in test_y.values]

        loss, acc = self.model.evaluate(test_x, test_y, verbose=2)
        scores = precision_recall_fscore_support(true, preds_select, average='macro')
        pre = scores[0]
        re = scores[1]
        f1 = scores[2]
        auc = np.nan
        try:
            auc = roc_auc_score(true, preds, multi_class="ovo")
        except:
            auc = np.nan

        report = classification_report(true, preds_select)
        cm = None #confusion_matrix(true, preds_select, labels=self.lables)

        if verbose:
            print("\nModel Results:")
            print("Loss: %f\nAccuracy: %f\nPrecision: %f\nRecall: %f\nF1: %f\nAUC: %f\n"%(loss, acc, pre, re, f1, auc))
            print("Classification Report:")
            print(report)
            # print("\nConfusion Matrix:")
            # print(tabulate([[k] + list(cm[i]) for i, k in enumerate(list(self.labels_to_values_map.keys()))], 
            #             ["Activity"] + list(label_map.keys()), 
            #             tablefmt="grid")
            # )

        return loss, acc, pre, re, f1, auc, preds, report, cm

## Model Archetype Handler

In [ ]:
class ModelArchetypeHandler():

    def __init__(self, archetype_name, experiment_name, source_labels, target_labels, save_dir="", checkpoint_path="checkpoints/", checkpoint_file_name="cp.ckpt"):
        self.archetype_name = archetype_name
        self.experiment_name = experiment_name
        self.source_labels = source_labels
        self.target_labels = target_labels
        self.save_dir = save_dir
        self.checkpoint_path = save_dir + checkpoint_path
        self.checkpoint_file_name = checkpoint_file_name
        self.hist = {}
        self.tracker = None
        self.names = ["naive", "initial_target", "transfer_target", "initial_comb", "transfer_comb"]

    # Build and Train Models ===================================================
    def run_experiment(self, input_shape_source, input_shape_target, train_layers, 
                       data, x_key,
                       n_labels_source=6, n_labels_target=6, learning_rate=0.001, metrics=["accuracy"], 
                       epochs=10, n_runs=5, 
                       verbose=True
                       ):
        
        tracker = []
        for run in range(n_runs):
            for n in self.names:
                if n not in self.hist:
                    self.hist.update({n: {}})
                if run not in self.hist[n]:
                    self.hist[n].update({run: None})

            # Naive model
            level = "naive"
            header = "\n\nRun %d, %s, %s ===="%(run, self.archetype_name, level)
            print(header.ljust(85, "="))
            self.n_model = ModelHandler(name="%s_%s_%d"%(self.archetype_name, level, run), 
                                        source_labels=self.source_labels, 
                                        target_labels=self.target_labels, 
                                        checkpoint_path="%s%s/%s/%d/"%(self.checkpoint_path, self.archetype_name, level, run), 
                                        checkpoint_file_name=self.checkpoint_file_name
                                        )
            self.n_model.build_model(self.archetype_name, input_shape_target, n_labels_target, learning_rate, metrics)
            self.n_model.train_model(level, 
                                     data["target"]["train"][x_key], data["target"]["train"]["y"], 
                                     data["target"]["val"][x_key], data["target"]["val"]["y"], 
                                     epochs*2
                                     )
            self.hist[level][run] = self.n_model.hist[level]
            for test in ["target"]:
                loss, acc, pre, re, f1, auc, preds, report, cm = self.n_model.evaluate(data[test]["test"][x_key], 
                                                                                       data[test]["test"]["y"], 
                                                                                       verbose=verbose
                                                                                       )
                tracker.append([self.archetype_name, level, run, test, loss, acc, pre, re, f1, auc, report, cm, preds, self.n_model.hist[level]])

            # Initial target model
            level = "initial_target"
            header = "\n\nRun %d, %s, %s ===="%(run, self.archetype_name, level)
            print(header.ljust(85, "="))
            self.t_target_model = ModelHandler(name="%s_%s_%d"%(self.archetype_name, level, run),
                                               source_labels=self.source_labels, 
                                               target_labels=self.target_labels, 
                                               checkpoint_path="%s%s/%s/%d/"%(self.checkpoint_path, self.archetype_name, level, run), 
                                               checkpoint_file_name=self.checkpoint_file_name
                                               )
            self.t_target_model.build_model(self.archetype_name, input_shape_source, n_labels_source, learning_rate, metrics)
            self.t_target_model.train_model(level, 
                                            data["source"]["train"][x_key], data["source"]["train"]["y"], 
                                            data["source"]["val"][x_key], data["source"]["val"]["y"], 
                                            epochs
                                            )
            self.hist[level][run] = self.t_target_model.hist[level]
            for test in ["source"]:
                loss, acc, pre, re, f1, auc, preds, report, cm = self.t_target_model.evaluate(data[test]["test"][x_key], 
                                                                                              data[test]["test"]["y"], 
                                                                                              verbose=verbose
                                                                                              )
                tracker.append([self.archetype_name, level, run, test, loss, acc, pre, re, f1, auc, report, cm, preds, self.t_target_model.hist[level]])

            # Transfer target model
            level = "transfer_target"
            header = "\n\nRun %d, %s, %s ===="%(run, self.archetype_name, level)
            print(header.ljust(85, "="))
            self.t_target_model.build_transfer_model(train_layers, input_shape_target, n_labels_target)
            self.t_target_model.train_model(level, 
                                            data["target"]["train"][x_key], data["target"]["train"]["y"], 
                                            data["target"]["val"][x_key], data["target"]["val"]["y"], 
                                            epochs
                                            )
            self.hist[level][run] = self.t_target_model.hist[level]
            for test in ["target"]:
                loss, acc, pre, re, f1, auc, preds, report, cm = self.t_target_model.evaluate(data[test]["test"][x_key], 
                                                                                              data[test]["test"]["y"], 
                                                                                              verbose=verbose
                                                                                              )
                tracker.append([self.archetype_name, level, run, test, loss, acc, pre, re, f1, auc, report, cm, preds, self.t_target_model.hist[level]])
                
        # Get run averages -----------------------------------------------------

        # Set up averages collecter
        averages = {}
        for level in list(self.names):
            averages.update({level: {}})
            for test in ["source", "target"]:
                averages[level].update({test: [0, 0, 0, 0, 0, 0]})
        
        # Get scores
        for row in tracker:
            for i in range(4, 10):
                averages[row[1]][row[3]][i-4] += row[i]
        
        # Add average scores to tracker
        for level in list(self.names):
            for test in ["source", "target"]:
                avg = [self.archetype_name, level, -1, test, 
                       averages[level][test][0]/n_runs, 
                       averages[level][test][1]/n_runs, 
                       averages[level][test][2]/n_runs, 
                       averages[level][test][3]/n_runs, 
                       averages[level][test][4]/n_runs, 
                       averages[level][test][5]/n_runs, 
                       np.nan, np.nan, np.nan, np.nan
                       ]
                tracker.append(avg)

        # To dataframe
        self.tracker = sorted(tracker, key=lambda x: (x[0], x[1], x[2], x[3]))
        self.tracker = pd.DataFrame(self.tracker, columns=["model_archetype", "model_type", 
                                                 "run", "test_level", "loss", 
                                                 "accuracy", "precision", 
                                                 "recall", "f1", "auc", "report", "confusion_matrix", 
                                                 "predictions", "history"
                                                 ])
        self.tracker["exp_name"] = [self.experiment_name] * self.tracker.shape[0]
        self.tracker.to_csv(self.save_dir + "trackers/" + self.experiment_name + "_" + self.archetype_name + ".csv", index=False)

        with open(self.save_dir + "histories/" + self.experiment_name + "_dense.pickle", "wb") as file:
            pickle.dump(self.hist, file)
        
        return self.tracker

## Dual Model Archetype Handler

In [ ]:
class DualModelArchetypeHandler():

    def __init__(self, experiment_name, source_labels, target_labels, save_dir="", checkpoint_path="checkpoints/", checkpoint_file_name="cp.ckpt"):
        self.experiment_name = experiment_name
        self.save_dir = save_dir
        self.checkpoint_path = checkpoint_path
        self.checkpoint_file_name = checkpoint_file_name

        self.dense_handler = ModelArchetypeHandler("dense", self.experiment_name, source_labels, target_labels, self.save_dir, self.checkpoint_path, self.checkpoint_file_name)
        self.conv_handler = ModelArchetypeHandler("conv", self.experiment_name, source_labels, target_labels, self.save_dir, self.checkpoint_path, self.checkpoint_file_name)
        
    def run_experiment(self, dense_input_shape_source, dense_input_shape_target, 
                       conv_input_shape_source, conv_input_shape_target, 
                       dense_train_layers, conv_train_layers, 
                       data, n_labels_source=6, n_labels_target=6, learning_rate=0.001, metrics=["accuracy"], 
                       epochs=10, n_runs=5, 
                       verbose=True
                       ):
        
        self.dense_handler.run_experiment(input_shape_source=dense_input_shape_source, 
                                          input_shape_target=dense_input_shape_target,
                                          train_layers=dense_train_layers, 
                                          data=data, x_key="x",
                                          n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                          learning_rate=learning_rate, metrics=metrics, 
                                          epochs=epochs, n_runs=n_runs, 
                                          verbose=verbose
                                          )
        
        self.conv_handler.run_experiment(input_shape_source=conv_input_shape_source, 
                                         input_shape_target=conv_input_shape_target, 
                                         train_layers=conv_train_layers, 
                                         data=data, x_key="x_shaped",
                                         n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                         learning_rate=learning_rate, metrics=metrics, 
                                         epochs=epochs, n_runs=n_runs, 
                                         verbose=verbose
                                         )
        
        self.dual_tracker = pd.concat([self.dense_handler.tracker, self.conv_handler.tracker])
        self.dual_tracker.to_csv(self.save_dir + "trackers/" + self.experiment_name + "_dual" + ".csv", index=False)
        
        with open(self.save_dir + "histories/" + exp_name + "_dense.pickle", "wb") as file:
            pickle.dump(self.dense_handler.hist, file)
        with open(self.save_dir + "histories/" + exp_name + "_conv.pickle", "wb") as file:
            pickle.dump(self.conv_handler.hist, file)

        return self.dual_tracker

# Plotting

## History

In [ ]:
def plot_histories(dual_handler, n_runs, epochs, save=False, save_dir="", save_name=""):
    fig, ax = plt.subplots(4, 4, figsize=(15, 15))
    # col 0: accuracy
    # col 1: val accuracy
    # col 2: loss
    # col 3: val loss
    # row 0: dense naive
    # row 1: dense transfer target val
    # row 2: conv naive
    # row 3: conv transfer target val

    # Subplot labels ===========================================================
    for i, architecture in enumerate(["Dense", "Conv."]):
        for k, model in enumerate(["Naive", "Transfer Target"]): #, "Transfer Combined"]):
            for j, metric in enumerate(["Accuracy", "Val. Accuracy", "Loss", "Val. Loss"]):
                row = k
                # if i > 0:
                #     row += 2
                ax[row][j].set_xlabel("Epochs")
                ax[row][j].set_ylabel(metric)
                ax[row][j].set_title(" ".join([architecture, model, metric]))

    # Subplots =================================================================
    labels = []
    avgs = {"accuracy": None, 
            "val_accuracy": None, 
            "loss": None, 
            "val_loss": None
            }
    for run in range(n_runs):
        labels.append("Run %d"%(run + 1))
        for j, score in enumerate(["accuracy", "val_accuracy", "loss", "val_loss"]):

            # Dense
            ax[0][j].plot(dual_handler.dense_handler.hist["naive"][run].history[score])
            ax[1][j].plot(dual_handler.dense_handler.hist["initial_target"][run].history[score] + dual_handler.dense_handler.hist["transfer_target"][run].history[score])
            # ax[2][j].plot(dual_handler.dense_handler.hist["initial_comb"][run].history[score] + dual_handler.dense_handler.hist["transfer_comb"][run].history[score])

            # Conv
            ax[2][j].plot(dual_handler.conv_handler.hist["naive"][run].history[score])
            ax[3][j].plot(dual_handler.conv_handler.hist["initial_target"][run].history[score] + dual_handler.conv_handler.hist["transfer_target"][run].history[score])
            # ax[5][j].plot(dual_handler.conv_handler.hist["initial_comb"][run].history[score] + dual_handler.conv_handler.hist["transfer_comb"][run].history[score])

    # Plot limits ==============================================================

    # Get values ---------------------------------------------------------------
    loss = [None, None]
    for i in range(4):
        for j in range(2):

            # Loss min
            if loss[0] is None:
                loss[0] = ax[i][j+2].get_ylim()[0]
            elif loss[0] > ax[i][j+2].get_ylim()[0]:
                loss[0] = ax[i][j+2].get_ylim()[0]

            # Loss max
            if loss[1] is None:
                loss[1] = ax[i][j+2].get_ylim()[1]
            elif loss[1] < ax[i][j+2].get_ylim()[1]:
                loss[1] = ax[i][j+2].get_ylim()[1]
                
    # Set values ---------------------------------------------------------------
    for i in range(4):
        for j in range(2):
            ax[i][j].set_ylim((0, 1))
            ax[i][j+2].set_ylim(loss)
    plt.setp(ax, xlim=(0, epochs*2-1))

    # Plot =====================================================================
    fig.legend(labels=labels, loc="upper right")
    plt.tight_layout()
    plt.subplots_adjust(right=0.9)
    if save:
        plt.savefig(save_dir + save_name)
    plt.plot()

## Tracker Results

In [ ]:
def plot_scores(tracker, save=False, save_dir="", save_name=""):
    fig, ax = plt.subplots(2, 2, figsize=(15, 10))

    metrics = [["precision", "recall"], ["accuracy", "f1"]]
    types = ["naive", "initial_target", "transfer_target"] #, "initial_comb", "transfer_comb"]
    test_sets = ["target"]
    archs = ["dense", "conv"]
    colors = [["xkcd:light orange", "xkcd:orange"], ["xkcd:sky blue", "xkcd:deep sky blue"]]
    label_keys = []

    for a in archs:
        for t in test_sets:
            label_keys.append(a + " " + t)

    # Get scores
    for i in range(2):
        for j in range(2):
            m = metrics[i][j]
            df = tracker.loc[tracker["run"] == -1]

            w = 0.2
            x_ticks = {0: np.arange(len(types))}
            c = 0
            for k, a in enumerate(archs):
                for l, t in enumerate(test_sets):
                    tmp = df.loc[(df["test_level"] == t) & (df["model_archetype"] == a)]
                    vals = tmp[m].tolist()
                    print(vals)
                    vals = [vals[2], vals[1], vals[4], vals[0], vals[3]]
                    print(1)
                    idx = label_keys.index("%s %s"%(a, t))
                    print(2)
                    ax[i][j].bar(x_ticks[c], vals, width=w) #, color=colors[k][l], label=label_keys[idx])
                    print(3)
                    c += 1
                    x_ticks.update({c: [x + w for x in x_ticks[c-1]]})

            print(4)
            ax[i][j].set_xlabel("Model Type")
            print(5)
            ax[i][j].set_ylabel("Score")
            print(6)
            ax[i][j].set_title(m.capitalize())
            print(7)

    plt.setp(ax, xticks=[x + w for x in range(len(types))], xticklabels=types)
    print(8)
    plt.tight_layout()
    print(9)
    plt.subplots_adjust(right=0.9)
    print(10)
    fig.legend(labels=label_keys, loc="upper right")
    print(11)
    if save:
        plt.savefig(save_dir + save_name)
    plt.show()

# Global Hyperparameters

In [ ]:
# File Handling
save_dir = "/content/drive/MyDrive/Classes/CSCE 5280 AI for Wearables/Group Project 2/Results/"      # Mica dir
checkpoint_path = "checkpoints/exp_"
cp_file = "checkpoint.ckpt"
hist_plot_dir = save_dir + "hist_plots/"
tracker_bar_dir = save_dir + "tracker_bar_plots/"

# Image shapes
img_rows_source = 33
img_rows_target = 27
img_cols_source = 17
img_cols_target = 15

# Model hyperparmeters
source_labels = sorted(list(source_labels))
target_labels = sorted(list(target_labels))
n_labels_source = len(source_labels) + 1
n_labels_target = len(target_labels) + 1
input_shape_dense_source = (561, )
input_shape_dense_target = (405, )
input_shape_conv_source = (img_rows_source, img_cols_source, 1)
input_shape_conv_target = (img_rows_target, img_cols_target, 1)
epochs = 20
n_runs = 5

# History
tracker_global = {}

# Experiments - Hyperparameter Tuning

## Experiment 1 - Baseline

### Hyperparameters

In [ ]:
exp_name = "001_baseline"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 1

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )



Run 0, dense, naive ===============================================================
Epoch 1/40
119/120 [============================>.] - ETA: 0s - loss: 0.8113 - accuracy: 0.7287
Epoch 1: val_loss improved from inf to 0.55178, saving model to /content/drive/MyDrive/Classes/CSCE 5280 AI for Wearables/Group Project 2/Results/checkpoints/exp_001_baseline/dense/naive/0/checkpoint.ckpt
120/120 [==============================] - 3s 19ms/step - loss: 0.8105 - accuracy: 0.7287 - val_loss: 0.5518 - val_accuracy: 0.7966
Epoch 2/40
117/120 [============================>.] - ETA: 0s - loss: 0.4095 - accuracy: 0.8443
Epoch 2: val_loss improved from 0.55178 to 0.29735, saving model to /content/drive/MyDrive/Classes/CSCE 5280 AI for Wearables/Group Project 2/Results/checkpoints/exp_001_baseline/dense/naive/0/checkpoint.ckpt
120/120 [==============================] - 2s 17ms/step - loss: 0.4105 - accuracy: 0.8447 - val_loss: 0.2974 - val_accuracy: 0.8805
Epoch 3/40
119/120 [========================

KeyboardInterrupt: ignored

In [ ]:
t

## Experiment 2 - Drop Learning Rate 1

### Hyperparameters

In [ ]:
exp_name = "002_drop_learning_rate_1"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.0001
dense_train_layers = 1

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )

In [ ]:
t

## Experiment 3 - Drop Learning Rate 2

### Hyperparameters

In [ ]:
exp_name = "003_drop_learning_rate_2"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.00001
dense_train_layers = 1

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )

In [ ]:
t

## Experiment 4 - Raise Learning Rate 1

### Hyperparameters

In [ ]:
exp_name = "004_raise_learning_rate_1"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.01
dense_train_layers = 1

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )

In [ ]:
t

## Experiment 5 - Raise Number of Unforzen Layers by 1

### Hyperparameters

In [ ]:
exp_name = "005_raise_num_unfrozen_1"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 2

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )
t

## Experiment 6 - Raise Number of Unforzen Layers by 2

### Hyperparameters

In [ ]:
exp_name = "006_raise_num_unfrozen_2"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 3

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )
t

## Experiment 7 - Raise Number of Unforzen Layers by 3

### Hyperparameters

In [ ]:
exp_name = "007_raise_num_unfrozen_3"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 4

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )
t

## Experiment 8 - Raise Number of Unforzen Layers by 4


### Hyperparameters

In [ ]:
exp_name = "008_raise_num_unfrozen_4"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 5

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )
t

## Experiment 9 - Raise Number of Unforzen Layers by 5

### Hyperparameters

In [ ]:
exp_name = "009_raise_num_unfrozen_5"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 6

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )
t

## Experiment 10 - Raise Number of Unforzen Layers by 6

### Hyperparameters

In [ ]:
exp_name = "010_raise_num_unfrozen_6"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 7

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )
t

## Experiment 11 - Raise Number of Unforzen Layers by 7

### Hyperparameters

In [ ]:
exp_name = "011_raise_num_unfrozen_7"

# File handling
exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Training
learning_rate = 0.001
dense_train_layers = 8

### Run Experiment

In [ ]:
single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                         checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                         )
t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                            dense_train_layers, 
                            data=data, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                            learning_rate=learning_rate, 
                            epochs=epochs, n_runs=n_runs, 
                            verbose=False
                            )
t

# Experiments - Scores vs Data

Hyperparameters:
* Learning Rate: 0.001
* train_layers = 6

## Base Hyperparameters

In [ ]:
exp_name_base = "_data_vs_score_keep_"
c = 11

# File handling
# exp_checkpoint_path = checkpoint_path + exp_name + "/"

# Data
keep = 0

# Training
learning_rate = 0.001
dense_train_layers = 6

## Select Data Function

In [ ]:
def select_data(data, keep):
    data_new = {"source": deepcopy(data["source"]), "target": {}}
    c = {}
    drops = []
    for i in range(1, 7):
        c.update({i: 0})

    for i, row in data["target"]["train"]["y"].iterrows():
        x = row["label"]

        if c[x] == keep:
            drops.append(i)
        else:
            c[x] += 1

    for k in ["val", "test"]:
        data_new["target"].update({k: deepcopy(data["target"][k])})

    data_new["target"].update({"train": {}})
    for k in ["x", "y"]:
        data_new["target"]["train"].update({k: data["target"]["train"][k].drop(drops)})

    return data_new

## Get Max Keep Count

In [ ]:
low = 100000
for i in range(1, 7):
    tmp = data["target"]["train"]["y"].loc[data["target"]["train"]["y"]["label"] == i]
    if tmp.size < low:
        low = tmp.size
    print(i, tmp.size, low)

## Experiments

### Experiment 12 - Keep 5

In [ ]:
# Hyperparameters
c = 12
keep = 5
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 13 - Keep 10

In [ ]:
# Hyperparameters
c = 13
keep= 10
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 14 - Keep 15

In [ ]:
# Hyperparameters
c = 14
keep = 15
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 15 - Keep 20

In [ ]:
# Hyperparameters
c = 15
keep = 20
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 16 - Keep 25

In [ ]:
# Hyperparameters
c = 16
keep = 25
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 17 - Keep 30

In [ ]:
# Hyperparameters
c = 17
keep = 30
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 18 - Keep 35

In [ ]:
# Hyperparameters
c = 18
keep = 35
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 19 - Keep 40

In [ ]:
# Hyperparameters
c = 19
keep = 40
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 20 - Keep 45

In [ ]:
 with open(save_dir + "histories/" + single_handler.experiment_name + "_dense.pickle", "wb") as file:
    pickle.dump(single_handler.hist, file)

In [ ]:
# Hyperparameters
c = 20
keep = 45
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 21 - Keep 50

In [ ]:
# Hyperparameters
c = 21
keep = 50
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 22 - Keep 55

In [ ]:
# Hyperparameters
c = 22
keep = 55
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 23 - Keep 60

In [ ]:
# Hyperparameters
c = 23
keep = 60
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 24 - Keep 65

In [ ]:
# Hyperparameters
c = 24
keep = 65
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 25 - Keep 70

In [ ]:
# Hyperparameters
c = 25
keep = 70
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 26 - Keep 75

In [ ]:
# Hyperparameters
c = 26
keep= 75
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 27 - Keep 80

In [ ]:
# Hyperparameters
c = 27
keep = 80
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 28 - Keep 85

In [ ]:
# Hyperparameters
c = 28
keep = 85
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 29 - Keep 90

In [ ]:
# Hyperparameters
c = 29
keep = 90
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 30 - Keep 95

In [ ]:
# Hyperparameters
c = 30
keep = 95
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 31 - Keep 100

In [ ]:
# Hyperparameters
c = 31
keep = 100
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 32 - Keep 105

In [ ]:
# Hyperparameters
c = 32
keep = 105
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 33 - Keep 110

In [ ]:
# Hyperparameters
c = 33
keep = 110
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 34 - Keep 115

In [ ]:
# Hyperparameters
c = 34
keep = 115
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 35 - Keep 120

In [ ]:
# Hyperparameters
c = 35
keep = 120
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 36 - Keep 125

In [ ]:
# Hyperparameters
c = 36
keep = 125
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 37 - Keep 130

In [ ]:
# Hyperparameters
c = 37
keep = 130
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 38 - Keep 135

In [ ]:
# Hyperparameters
c = 38
keep = 135
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 39 - Keep 140

In [ ]:
# Hyperparameters
c = 39
keep = 140
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 40 - Keep 145

In [ ]:
# Hyperparameters
c = 40
keep = 145
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 41 - Keep 150

In [ ]:
# Hyperparameters
c = 41
keep = 150
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 42 - Keep 155

In [ ]:
# Hyperparameters
c = 42
keep = 155
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

### Experiment 43 - Keep 160

In [ ]:
# Hyperparameters
c = 43
keep = 160
exp_name = str(c) + exp_name_base + str(keep)
exp_checkpoint_path = checkpoint_path + exp_name + "/"
print(exp_name)

# Data
if low >= keep:
    data_new = select_data(data, keep)

    # Train
    single_handler = ModelArchetypeHandler("dense", exp_name, source_labels, target_labels, save_dir=save_dir, 
                                            checkpoint_path=exp_checkpoint_path, checkpoint_file_name=cp_file
                                            )
    t = single_handler.run_experiment(input_shape_dense_source, input_shape_dense_target, 
                                dense_train_layers, 
                                data=data_new, x_key="x", n_labels_source=n_labels_source, n_labels_target=n_labels_target, 
                                learning_rate=learning_rate, 
                                epochs=epochs, n_runs=n_runs, 
                                verbose=False
                                )
    t

https://stackoverflow.com/questions/73029592/transfer-learning-from-a-custom-model-trained-with-different-image-sizes

# Bottom